In [1]:
%%capture
import pandas as pd
import outcomes_extraction as ke
import difflib
from ast import literal_eval
from pprint import pprint
import numpy as np
from scipy.spatial.distance import cosine

import warnings
warnings.filterwarnings(action="ignore")

### Загрузка данных

In [44]:
# сущности из БД
items = pd.read_csv("dataprocessing_items_202305252034.csv")
items["lowercase"] = items.name.str.lower()

# данные по дисциплинам из БД
df = pd.read_csv("disc_contents20230506_4.csv")

# кластеры сущностей
clusters = pd.read_csv("clusters_of_disciplines.csv")
clusters = clusters["nodes"].to_dict()

# кластеры по заново размеченному графу
updated_clusters = pd.read_csv("updated_graph_clusters.csv")
updated_clusters = updated_clusters["nodes"].to_dict()

### Обработка данных

In [12]:
from gensim.models import Word2Vec, KeyedVectors
import gensim.parsing.preprocessing as pp
import re
from pymystem3 import Mystem
import pandas as pd
import numpy as np
from ast import literal_eval
from scipy.spatial.distance import cosine

m = Mystem()

In [41]:
def get_entities(text, entities_to_match=150):
    """Здесь сущности извлекаются из текста и памятся с сущностями из БД
    150 - это вообще слишком много, но так сущности берутся по максимуму, 
    отсюда примерно 3-6 секунд на выполнение извлечения
    Дальше то, что извлеклось, мапится с БД без ограничений по количеству"""
    
    example = ke.simple_outcomes_extraction(text, n_best=entities_to_match)
    res = pd.DataFrame(columns=["id", "name"])
    for word in example:
        close = difflib.get_close_matches(word, items.lowercase.tolist(), cutoff=0.85, n=1)
        if close:
            res = res.append(items[items.lowercase.isin(close)][["id", "name"]])
            res.drop_duplicates(subset=["name"], inplace=True)
    return res.name.tolist()



class Discipline:

    """Векторизация дисциплины по набору сущностей"""

    def __init__(self, entities, clusters):
        self.entities = entities
        self.n_entities = len(self.entities)
        self.clusters = clusters


    @property
    def vectorized_discipline(self):
        """векторизация дисциплины"""
        vector = np.zeros(len(self.clusters))
        for ent in self.entities:
            for cluster, values in self.clusters.items():
                if ent in values:
                    vector[cluster]+=1
        return vector


    @property
    def norm_vector(self):
        """нормализация вектора"""
        x = self.vectorized_discipline
        return (x-np.min(x))/(np.max(x)-np.min(x))
    

    @property
    def n_domains(self):
        return np.count_nonzero(self.vectorized_discipline)
    


class EntityContext:

    """Векторизация дисциплины по набору сущностей"""

    def __init__(self, text, model):
        self.text = text
        self.model = model


    def input_preprocessing(self):
        text = pp.strip_multiple_whitespaces(re.sub(r'[«»]', ""," ".join(pp.preprocess_string(self.text))))
        text = m.lemmatize(text)
        text = [val for val in text if not val.isspace()]
        return text


    @property
    def norm_vector(self):
        """векторизация дисциплины"""
        tokens = self.input_preprocessing()
        vectors = [self.model.wv.get_vector(token) for token in tokens if token in self.model.wv.index_to_key]
        vec = np.mean(vectors, axis=0) if len(vectors) != 0 else np.array([0]*300)
        return vec



### Тестирование

В датафрейме `df` можно найти по столбцу `id` данные дисциплины в Конструкторе. Значение для `id` берется из ссылки. В примере: https://op.itmo.ru/work-program/2856/general. В столбце `comb_res` хранятся учебные сущности дисциплины в виде списка.

In [4]:
discipline = df.query("id == 2856").iloc[0]
print(discipline.title)
entities = literal_eval(discipline.comb_res)
entities

Web-программирование


['Современные технологии разработки',
 'Разработка серверной части приложения',
 'Разработка интерфейсов web-приложений',
 'Разработка Single Page Application и Rich Internet Application',
 'vue.js',
 'django web framework',
 'django REST framework',
 'Сетевая модель OSI',
 'Адресация в сетях IP',
 'Модели данных',
 'Работа с токенами',
 'Паттерны проектирования']

Если дисциплины в файлике нет или это сгенеренный текст, то его можно его прямо текстом с разметкой передать в функцию `get_entities`.

In [5]:
TEXT = """- Основы Web-программирования
    - Основы HTML/CSS
        - Структура веб-страницы
        - Основы стилизации
    - JavaScript и фреймворки
        - Основы программирования на JavaScript
        - Введение в фреймворки JavaScript

- Разработка Backend и Базы Данных
    - Работа с базами данных
        - Основы SQL
        - Принципы проектирования баз данных
    - Backend разработка
        - Основы серверной логики
        - Программирование на Node.js

- Дизайн и Пользовательский Интерфейс
    - Принципы дизайна и UX/UI
        - Основы дизайна веб-интерфейсов
        - Основы UX/UI

- Тестирование и Безопасность
    - Тестирование и отладка
        - Методы тестирования веб-приложений
        - Инструменты отладки
    - Безопасность веб-приложений
        - Основы кибербезопасности
        - Защита от веб-угроз

- Продвинутые Темы
    - Разработка мобильных приложений
        - Основы мобильной веб-разработки
        - Адаптивный дизайн
    - Инструменты разработки и среды программирования
        - Работа с интегрированными средами разработки (IDE)
        - Версионный контроль
    - Разработка API
        - Принципы RESTful API
        - GraphQL
    - Работа с версионным контролем (Git)
        - Основы Git
        - Работа с ветками и слияниями
"""


entities_gen = get_entities(TEXT)
entities_gen

['Основы программирования',
 'Веб-приложение',
 'Базы данных',
 'Основы кибербезопасности',
 'Пользовательский интерфейс',
 'Методики тестирования',
 'Принципы проектирования ПО',
 'веб-страница',
 'Веб-страница',
 'RESTful APIs',
 'Данные',
 '\ufeffВеб-разработка',
 'Программирование',
 'Разработка мобильных приложений',
 'Проектирование БД',
 'Программирование роботов',
 'Разработка ПО',
 'Среда разработки VBA',
 'Основы дизайна',
 'Проектирование баз данных',
 'Адаптивный дизайн',
 'Кибербезопасность',
 'Фреймворки',
 'Мобильное приложение']

### Сравнение эмбеддингов

In [6]:
# тестируем векторизацию
print(f"Дисциплина: {discipline['title']}")

subj = Discipline(entities, clusters)

print(f"Количество сущностей у дисциплины: {subj.n_entities}")
pprint(subj.entities)

print(f"Количество предметных областей: {subj.n_domains}")
print(subj.vectorized_discipline)

print("Нормализованный вектор")
print(subj.norm_vector)

Дисциплина: Web-программирование
Количество сущностей у дисциплины: 12
['Современные технологии разработки',
 'Разработка серверной части приложения',
 'Разработка интерфейсов web-приложений',
 'Разработка Single Page Application и Rich Internet Application',
 'vue.js',
 'django web framework',
 'django REST framework',
 'Сетевая модель OSI',
 'Адресация в сетях IP',
 'Модели данных',
 'Работа с токенами',
 'Паттерны проектирования']
Количество предметных областей: 5
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 2. 8. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
Нормализованный вектор
[0.    0.    0.    0.    0.    0.    0.    0.125 0.    0.    0.    0.
 0.    0.25  1.    0.    0.    0.125 0.    0.    0.    0.    0.  

In [7]:
# сравнение двух дисциплин
subj2 = Discipline(entities_gen, clusters)
print(np.round(1-cosine(subj2.norm_vector, subj.norm_vector), 3))


0.618


## Сравнение эмбеддингов по размеченному графу

В нем примерно в 2 раза меньше связей.

In [45]:
# тестируем векторизацию по размеченному графу
print(f"Дисциплина: {discipline['title']}")

subj = Discipline(entities, updated_clusters)

print(f"Количество сущностей у дисциплины: {subj.n_entities}")
print(subj.entities)

print(f"Количество предметных областей: {subj.n_domains}")
print(subj.vectorized_discipline)

print("Нормализованный вектор")
print(subj.norm_vector)

Дисциплина: Web-программирование
Количество сущностей у дисциплины: 12
['Современные технологии разработки', 'Разработка серверной части приложения', 'Разработка интерфейсов web-приложений', 'Разработка Single Page Application и Rich Internet Application', 'vue.js', 'django web framework', 'django REST framework', 'Сетевая модель OSI', 'Адресация в сетях IP', 'Модели данных', 'Работа с токенами', 'Паттерны проектирования']
Количество предметных областей: 1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Нормализованный вектор
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 

In [46]:
# сравнение двух дисциплин
subj2 = Discipline(entities_gen, updated_clusters)
print(f"Количество сущностей у дисциплины: {subj2.n_entities}")
print(subj2.entities)

print(f"Количество предметных областей: {subj2.n_domains}")
print(subj2.vectorized_discipline)

print("Нормализованный вектор")
print(subj2.norm_vector)
print("Сходство эмбеддингов:")
print(np.round(1-cosine(subj2.norm_vector, subj.norm_vector), 3))

Количество сущностей у дисциплины: 24
['Основы программирования', 'Веб-приложение', 'Базы данных', 'Основы кибербезопасности', 'Пользовательский интерфейс', 'Методики тестирования', 'Принципы проектирования ПО', 'веб-страница', 'Веб-страница', 'RESTful APIs', 'Данные', '\ufeffВеб-разработка', 'Программирование', 'Разработка мобильных приложений', 'Проектирование БД', 'Программирование роботов', 'Разработка ПО', 'Среда разработки VBA', 'Основы дизайна', 'Проектирование баз данных', 'Адаптивный дизайн', 'Кибербезопасность', 'Фреймворки', 'Мобильное приложение']
Количество предметных областей: 1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Нормализованный вектор
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

Получилось, что в обоих случаях все сущности дисциплины принадлежат к одному кластеру, но это разные кластеры.

## Сравнение родных контекстных эмбеддингов

In [42]:
model = Word2Vec.load("w2v_rpd.bin")

print(f"Дисциплина: {discipline['title']}")

subj = EntityContext(discipline["raw_text"], model)

print(subj.norm_vector)

Дисциплина: Web-программирование
[-0.07589009  0.30533388 -0.6385905  -0.3474625   0.03728777  0.21546368
  0.82632554  0.03575581  0.5062906   0.4852555   0.03743974 -0.19947794
 -0.06587341  0.05278833 -0.3189453  -0.10487264  0.15469797  0.07478844
 -0.07040863  0.43154553  0.1841274  -0.12739669  0.05362587 -0.12707227
  0.61998874 -0.39598837  0.40642697  0.05288816  0.38749725 -0.16116884
  0.46810928  0.2136413  -0.14680733  0.3808903  -0.0399343  -0.29508275
  0.4874148   0.05004387  0.16500369  0.4770348  -0.43545002  0.20675565
  0.35989884 -0.13342956  0.08000717  0.1377643   0.09140519  0.19696714
 -0.27686888 -0.05220878 -0.2827377   0.28201592  0.20056595  0.03684692
 -0.16985056 -0.2931888   0.19188248 -0.02097412  0.18773673  0.15421504
  0.11106184 -0.55085105  0.19482352 -0.49834073 -0.40081415  0.12142014
 -0.03497323 -0.22754131  0.243905   -0.67257303 -0.4232969  -0.04475499
  0.33677092  0.39774472 -0.42325956  0.10739956 -0.02806684 -0.31800166
  0.14654975  0.07

In [43]:
# сравнение двух дисциплин
subj2 = EntityContext(TEXT, model)

print(subj2.norm_vector)

print("Сходство эмбеддингов:")
print(np.round(1-cosine(subj2.norm_vector, subj.norm_vector), 3))

[-3.22332114e-01  5.27481079e-01 -1.26915872e+00 -2.93312728e-01
 -1.72859505e-01  1.89129338e-01  1.13494635e+00  4.28536564e-01
  8.62121642e-01  6.07096136e-01 -1.33131132e-01 -8.84447247e-02
 -3.47864442e-02  1.32472171e-02 -4.89405930e-01 -1.76617295e-01
  1.65763080e-01  9.58394259e-02  1.40732989e-01  8.08122635e-01
  5.68033218e-01 -1.02849297e-01  1.01913281e-01 -1.10365279e-01
  8.65695894e-01 -5.71803689e-01  6.25007093e-01  4.04593199e-01
  7.10693121e-01 -1.54916942e-02  5.87973535e-01  3.45928699e-01
  4.79819998e-02  6.33556664e-01 -5.42175956e-04 -2.11739004e-01
  9.04759943e-01  6.24824828e-03  1.22108616e-01  5.70263505e-01
 -6.95933878e-01  3.53823483e-01  5.60033858e-01  1.10540852e-01
  1.51532367e-01  8.64779130e-02  9.56494957e-02  7.54198283e-02
 -1.62902966e-01 -3.95352215e-01  5.05229495e-02  4.17447716e-01
  4.61373299e-01  1.24904513e-01 -3.79070163e-01 -3.37057531e-01
  4.29031521e-01 -2.71490626e-02  3.41584802e-01  3.93829703e-01
  9.68451053e-02 -4.37766